In [73]:
from functools import partial
from datasets import load_dataset
import torch
import torch.nn as nn
import numpy as np

In [57]:
class ArcEager:
    def __init__(self, sentence):
        """arc-eager transition system
        Args:
            sentence (list): list of words in the sentence
            
        Attributes:
            sentence (list): list of words in the sentence
            buffer (list): list of indexes of words in the sentence
            stack (list): list of indexes of words in the sentence
            arcs (list): list of indexes of words in the sentence
              
        
  """

        # sentence is the input for which we want to build our Arc-Standard
        self.sentence = sentence

        # here we create the buffer having an array of indexes with the same length as the sentence
        # basically, each word has its own index in this buffer
        # we have initialized the buffer having all the words in the sentence

        self.buffer = [i for i in range(len(self.sentence))] 

        # initialize the stack empty 
        
        self.stack = []

        # representation of the tree
        # every word will have a -1 assigned -> no father has been assigned yet

        self.arcs = [-1 for _ in range(len(self.sentence))]

        # three shift moves to initialize the stack

        # means that in the stack now is the ROOT
        # self.shift() it calls a method that implements this operation; we will look at it after 

        self.shift() 


    def shift(self):
     # if len(self.buffer) > 0:
       
      b1 = self.buffer[0]
      self.buffer = self.buffer[1:]
      self.stack.append(b1)
    
    def left_arc(self): 
     # if self.stack[-1] !=0 and self.arcs[self.stack[-1]]==-1:
      o1 = self.stack.pop()
      o2 = self.buffer[0]
      self.arcs[o1] = o2
 
    def right_arc(self):
     o1 = self.buffer[0]
     o2 = self.stack[-1]
     self.arcs[o1] = o2
     self.stack.append(o1)
     self.buffer = self.buffer[1:]

    def reduce(self):
      # if self.arcs[self.stack[-1]] != -1:
       self.stack.pop()

    
    def is_tree_final(self):
     return len(self.stack) == 1 and len(self.buffer) == 0
     
    def can_shift(self):
       result = False
       if len(self.buffer) > 0:
        return True
       return result
    def can_left_arc(self):
     result = False
     stack_top = self.stack[-1]
     if stack_top!=0 and self.arcs[stack_top]==-1:
      return True
     return result
  
    def can_right_arc(self):
     stack_len = len(self.stack)
     if stack_len > 2 and len(self.buffer) > 0:
      return True
     return False
  
    def can_reduce(self):
     result = False
     if self.arcs[self.stack[-1]] != -1:
      return True
     return result

    def print_configuration(self):

      s = [self.sentence[i] for i in self.stack]
      b = [self.sentence[i] for i in self.buffer]
      print(s,b)
      print(self.arcs)
    
      

In [58]:
sentence = ["<ROOT>", "He","began","to","write","again","."]
gold = [-1, 2, 0, 4, 2, 4, 2]

# sentence = ["<ROOT>", "He","wrote","her","a","letter","."]
# gold = [-1, 2, 0, 2, 5, 2, 2]

parser = ArcEager(sentence)
parser.print_configuration()

['<ROOT>'] ['He', 'began', 'to', 'write', 'again', '.']
[-1, -1, -1, -1, -1, -1, -1]


In [59]:
class Oracle:
  """
  Oracle class that implements the oracle for the arc-eager transition system
  
  Args:
    parser (ArcEager): the parser
    gold_tree (list): the gold tree
    
  Attributes:
    parser (ArcEager): the parser
    gold_tree (list): the gold tree
  """
  def __init__(self, parser, gold_tree):
    
    self.parser = parser
    self.gold = gold_tree

  def is_left_arc_gold(self):
    if len(self.parser.buffer) == 0:
      return False
    o1 = self.parser.stack[-1]
    o2 = self.parser.buffer[0]

    if self.gold[o1] == o2 and self.parser.arcs[o1] == -1 and o1 != -1:
      return True
    return False


  def is_right_arc_gold(self):
    if len(self.parser.buffer) == 0:
      return False
    o1 = self.parser.stack[-1]
    o2 = self.parser.buffer[0]

    if self.gold[o2] == o1:
      return True

    return False

  def is_shift_gold(self):
    if len(self.parser.buffer) == 0:
      return False
    if (self.is_left_arc_gold() or self.is_right_arc_gold() or self.is_reduce_gold()):
      return False

    return True

  def is_reduce_gold(self):
    if len(self.parser.stack) < 2: 
      return False
    stack_top = self.parser.stack[-1]
    if self.has_head(stack_top) and self.has_all_children(stack_top):
      return True
    return False

  def has_head(self, stack_top):
    return  self.parser.arcs[stack_top] != -1 


  def has_all_children(self, stack_top):
    i = 0
    for arc in self.gold:
      if arc == stack_top:
        if self.parser.arcs[i] != stack_top:
          return False
      i+=1
    return True
  
  def can_left_arc(self):
   result = False
   stack_top = self.parser.stack[-1]
   if stack_top!=0 and self.parser.arcs[stack_top]==-1:
    return True
   return result
  
  def can_right_arc(self):
   stack_len = len(self.parser.stack)
   if stack_len > 2 and len(self.parser.buffer) > 0:
     return True
   return False
  
  def can_reduce(self):
    result = False
    if self.parser.arcs[self.parser.stack[-1]] != -1:
      return True
    return result

In [60]:
sentence = ["<ROOT>", "He","began","to","write","again","."]
gold = [-1, 2, 0, 4, 2, 4, 2]

# sentence = ["<ROOT>", "He","wrote","her","a","letter","."]
# gold = [-1, 2, 0, 2, 5, 2, 2]

parser = ArcEager(sentence)
oracle = Oracle(parser, gold)

parser.print_configuration()

['<ROOT>'] ['He', 'began', 'to', 'write', 'again', '.']
[-1, -1, -1, -1, -1, -1, -1]


In [61]:
while not parser.is_tree_final():
    if oracle.is_left_arc_gold() and parser.can_left_arc():
        parser.left_arc()
    elif oracle.is_right_arc_gold():
        parser.right_arc()
    elif oracle.is_reduce_gold() and parser.can_reduce():
        parser.reduce()
    elif oracle.is_shift_gold() and parser.can_shift():
        parser.shift()
        
print(parser.arcs)
print(gold)
    

[-1, 2, 0, 4, 2, 4, 2]
[-1, 2, 0, 4, 2, 4, 2]


In [62]:
# returns whether a tree is projective or not

def is_projective(tree):
    for i in range(len(tree)):
        if tree[i]==-1:
            continue
        left = min(i, tree[i])
        right = max(i, tree[i])

        for j in range(0, left):
            if tree[j] > left and tree[j] < right:
                return False
        for j in range(left+1, right):
            if tree[j] < left or tree[j] > right:
                return False
        for j in range(right+1, len(tree)):
            if tree[j] > left  and tree[j] < right:
                return False
    return True


In [63]:
def create_dict(dataset, threshold = 3):

    """ ceate a dictionary of words with frequency >= threshold
    """
    
    dic = {}

    for sample in dataset:
        for word in sample['tokens']:
            if word in dic:
                dic[word] += 1
            else:
                dic[word] = 1
    
    map = {}
    map["<pad>"] = 0
    map["<ROOT>"] = 1
    map["<unk>"] = 2

    next_indx = 3
    for word in dic.keys():
        if dic[word] >= threshold:
            map[word] = next_indx
            next_indx += 1
    return map

In [64]:
train_dataset = load_dataset('universal_dependencies', 'en_lines', split = 'train')
dev_dataset = load_dataset('universal_dependencies', 'en_lines', split = 'validation')
test_dataset = load_dataset('universal_dependencies', 'en_lines', split = 'test')

# remove non-projective sentences: heads in the gold 
# tree are strings, we convert them to int

train_dataset =[sample for sample in train_dataset if is_projective([-1] + [int(head) for head in sample['head']])]

# create embedding dictionary

emb_dictionary = create_dict(train_dataset)


print("Number of samples:")
print("Train:\t", len(train_dataset))
print("Dev:\t", len(dev_dataset))
print("Test:\t", len(test_dataset))

Found cached dataset universal_dependencies (C:/Users/roven/.cache/huggingface/datasets/universal_dependencies/en_lines/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)
Found cached dataset universal_dependencies (C:/Users/roven/.cache/huggingface/datasets/universal_dependencies/en_lines/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)
Found cached dataset universal_dependencies (C:/Users/roven/.cache/huggingface/datasets/universal_dependencies/en_lines/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)


Number of samples:
Train:	 2922
Dev:	 1032
Test:	 1035


In [74]:
def process_sample(sample, get_gold_path = False):

  # put sentence and gold tree in our format
  sentence = ["<ROOT>"] + sample["tokens"]
  gold = [-1] + [int(i) for i in sample["head"]]  #heads in the gold tree are strings, we convert them to int
  
  # embedding ids of sentence words
  enc_sentence = [emb_dictionary[word] if word in emb_dictionary else emb_dictionary["<unk>"] for word in sentence]

  # gold_path and gold_moves are parallel arrays whose elements refer to parsing steps
  gold_path = []   # record two topmost stack tokens and first buffer token for current step
  gold_moves = []  # contains oracle (canonical) move for current step: 0 is left, 1 right, 2 shift

  if get_gold_path:  # only for training
    parser = ArcEager(sentence)
    oracle = Oracle(parser, gold)

    while not parser.is_tree_final():
      
      # save configuration
      configuration = [parser.stack[len(parser.stack)-2], parser.stack[len(parser.stack)-1]]
      if len(parser.buffer) == 0:
        configuration.append(-1)
      else:
        configuration.append(parser.buffer[0])  
      gold_path.append(configuration)

      # save gold move
      if oracle.is_left_arc_gold(): 
        gold_moves.append(0)
        parser.left_arc()
      elif oracle.is_right_arc_gold():
        parser.right_arc()
        gold_moves.append(1)
      elif oracle.is_shift_gold():
        parser.shift()
        gold_moves.append(2)
      elif oracle.is_reduce_gold():
        parser.reduce()
        gold_moves.append(3)

  return enc_sentence, gold_path, gold_moves, gold

    # gold_path stores the configurations of the stack and the buffer
    # gold_moves stores the correct gold move at each configuration

    


In [75]:
def prepare_batch(batch_data, get_gold_path = False):
    data = [process_sample(s, get_gold_path = get_gold_path) for s in batch_data]

    sentences = [s[0] for s in data]
    paths = [s[1] for s in data]
    moves = [s[2] for s in data]
    trees = [s[3] for s in data]

    return sentences, paths, moves, trees

In [76]:
BATCH_SIZE = 32

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size= BATCH_SIZE, shuffle = True, collate_fn = partial(prepare_batch, get_gold_path = True))
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size= BATCH_SIZE, shuffle = True, collate_fn = partial(prepare_batch))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size= BATCH_SIZE, shuffle = True, collate_fn = partial(prepare_batch))

In [77]:
# define hyperparameters of NN
EMBEDDING_SIZE = 100
LSTM_SIZE = 100
LSTM_LAYERS = 2
MLP_SIZE = 100
DROPOUT = 0.2
EPOCHS = 15
LR = 0.001

In [89]:
class Net(nn.Module):

  def __init__(self, device):
    super(Net, self).__init__()
    self.device = device
    self.embeddings = nn.Embedding(len(emb_dictionary), EMBEDDING_SIZE, padding_idx=emb_dictionary["<pad>"])

    # initialize bi-LSTM
    self.lstm = nn.LSTM(EMBEDDING_SIZE, LSTM_SIZE, num_layers = LSTM_LAYERS, bidirectional=True, dropout=DROPOUT)

    # initialize feedforward
    self.w1 = torch.nn.Linear(6*LSTM_SIZE, MLP_SIZE, bias=True)
    self.activation = torch.nn.Tanh()
    self.w2 = torch.nn.Linear(MLP_SIZE, 4, bias=True)
    self.softmax = torch.nn.Softmax(dim=-1)

    self.dropout = torch.nn.Dropout(DROPOUT)


  def forward(self, x, paths):
    # get the embeddings
    x = [self.dropout(self.embeddings(torch.tensor(i).to(self.device))) for i in x]

    # run the bi-lstm
    h = self.lstm_pass(x)

    # for each parser configuration that we need to score we arrange from the
    # output of the bi-lstm the correct input for the feedforward
    mlp_input = self.get_mlp_input(paths, h)

    # run the feedforward and get the scores for each possible action
    out = self.mlp(mlp_input)

    return out

  def lstm_pass(self, x):
    x = torch.nn.utils.rnn.pack_sequence(x, enforce_sorted=False)
    h, (h_0, c_0) = self.lstm(x)
    h, h_sizes = torch.nn.utils.rnn.pad_packed_sequence(h) # size h: (length_sentences, batch, output_hidden_units)
    return h

  def get_mlp_input(self, configurations, h):
    mlp_input = []
    zero_tensor = torch.zeros(2*LSTM_SIZE, requires_grad=False).to(self.device)
    for i in range(len(configurations)): # for every sentence in the batch
      for j in configurations[i]: # for each configuration of a sentence
        mlp_input.append(torch.cat([zero_tensor if j[0]==-1 else h[j[0]][i], zero_tensor if j[1]==-1 else h[j[1]][i], zero_tensor if j[2]==-1 else h[j[2]][i]]))
    mlp_input = torch.stack(mlp_input).to(self.device)
    return mlp_input

  def mlp(self, x):
    return self.softmax(self.w2(self.dropout(self.activation(self.w1(self.dropout(x))))))

  # we use this function at inference time. We run the parser and at each step
  # we pick as next move the one with the highest score assigned by the model
  def infere(self, x):

    parsers = [ArcEager(i) for i in x]

    x = [self.embeddings(torch.tensor(i).to(self.device)) for i in x]

    h = self.lstm_pass(x)

    while not self.parsed_all(parsers):
      # get the current configuration and score next moves
      configurations = self.get_configurations(parsers)
      mlp_input = self.get_mlp_input(configurations, h)
      mlp_out = self.mlp(mlp_input)
      # take the next parsing step
      self.parse_step(parsers, mlp_out)

    # return the predicted dependency tree
    return [parser.arcs for parser in parsers]

  def get_configurations(self, parsers):
    configurations = []

    for parser in parsers:
      if parser.is_tree_final():
        conf = [-1, -1, -1]
      else:
        conf = [parser.stack[len(parser.stack)-2], parser.stack[len(parser.stack)-1]]
        if len(parser.buffer) == 0:
          conf.append(-1)
        else:
          conf.append(parser.buffer[0])
      configurations.append([conf])

    return configurations

  def parsed_all(self, parsers):
    for parser in parsers:
      if not parser.is_tree_final():
        return False
    return True

  # In this function we select and perform the next move according to the scores obtained.
  # We need to be careful and select correct moves, e.g. don't do a shift if the buffer
  # is empty or a left arc if σ2 is the ROOT. For clarity sake we didn't implement
  # these checks in the parser so we must do them here. This renders the function quite ugly
  def parse_step(self, parsers, moves):
      moves_argm = moves.argmax(-1)
      for parser, move_arg in zip(parsers, moves_argm):
        if parser.is_tree_final():
          continue

        stack_len = len(parser.stack)
        buffer_len = len(parser.buffer)
        stack_top = parser.stack[-1]

        if move_arg == 0:
          if buffer_len > 0 and stack_top != 0:
            parser.left_arc()
          else:
            self.apply_default_move(parser, stack_len, buffer_len)
        elif move_arg == 1:
          if stack_len >= 2 and buffer_len > 0:
            parser.right_arc()
          else: 
            self.apply_default_move(parser, stack_len, buffer_len)
        elif move_arg == 2:
          if buffer_len > 0:
            parser.shift()
          else:
            self.apply_default_move(parser, stack_len, buffer_len)
        elif move_arg == 3:
          if stack_len >= 2:
            parser.reduce()
          else:
            self.apply_default_move(parser, stack_len, buffer_len)

  def apply_default_move(self, parser, stack_len, buffer_len):
    stack_top = parser.stack[-1]
    if buffer_len > 0:
      if stack_top != 0:
        parser.left_arc()
      else:
        parser.shift()
    elif stack_len >= 2:
      parser.reduce()
    else:
      parser.shift()

In [81]:
"""

class Net(nn.Module):

  def __init__(self, device):
    super(Net, self).__init__()
    self.device = device
    self.embeddings = nn.Embedding(len(emb_dictionary), EMBEDDING_SIZE, padding_idx=emb_dictionary["<pad>"])

    # initialize bi-LSTM
    self.lstm = nn.LSTM(EMBEDDING_SIZE, LSTM_SIZE, num_layers = LSTM_LAYERS, bidirectional=True, dropout=DROPOUT)

    # initialize feedforward
    self.w1 = torch.nn.Linear(6*LSTM_SIZE, MLP_SIZE, bias=True)
    self.activation = torch.nn.Tanh()
    self.w2 = torch.nn.Linear(MLP_SIZE, 4, bias=True)
    self.softmax = torch.nn.Softmax(dim=-1)

    self.dropout = torch.nn.Dropout(DROPOUT)


  def forward(self, x, paths):
    # get the embeddings
    x = [self.dropout(self.embeddings(torch.tensor(i).to(self.device))) for i in x]

    # run the bi-lstm
    h = self.lstm_pass(x)

    # for each parser configuration that we need to score we arrange from the
    # output of the bi-lstm the correct input for the feedforward
    mlp_input = self.get_mlp_input(paths, h)

    # run the feedforward and get the scores for each possible action
    out = self.mlp(mlp_input)

    return out

  def lstm_pass(self, x):
    x = torch.nn.utils.rnn.pack_sequence(x, enforce_sorted=False)
    h, (h_0, c_0) = self.lstm(x)
    h, h_sizes = torch.nn.utils.rnn.pad_packed_sequence(h) # size h: (length_sentences, batch, output_hidden_units)
    return h

  def get_mlp_input(self, configurations, h):
    mlp_input = []
    zero_tensor = torch.zeros(2*LSTM_SIZE, requires_grad=False).to(self.device)
    for i in range(len(configurations)): # for every sentence in the batch
      for j in configurations[i]: # for each configuration of a sentence
        mlp_input.append(torch.cat([zero_tensor if j[0]==-1 else h[j[0]][i], zero_tensor if j[1]==-1 else h[j[1]][i], zero_tensor if j[2]==-1 else h[j[2]][i]]))
    mlp_input = torch.stack(mlp_input).to(self.device)
    return mlp_input

  def mlp(self, x):
    return self.softmax(self.w2(self.dropout(self.activation(self.w1(self.dropout(x))))))

  # we use this function at inference time. We run the parser and at each step
  # we pick as next move the one with the highest score assigned by the model
  def infere(self, x):

    parsers = [ArcEager(i) for i in x]

    x = [self.embeddings(torch.tensor(i).to(self.device)) for i in x]

    h = self.lstm_pass(x)

    while not self.parsed_all(parsers):
      # get the current configuration and score next moves
      configurations = self.get_configurations(parsers)
      mlp_input = self.get_mlp_input(configurations, h)
      mlp_out = self.mlp(mlp_input)
      # take the next parsing step
      self.parse_step(parsers, mlp_out)

    # return the predicted dependency tree
    return [parser.arcs for parser in parsers]

  def get_configurations(self, parsers):
    configurations = []

    for parser in parsers:
      if parser.is_tree_final():
        conf = [-1, -1, -1]
      else:
        conf = [parser.stack[len(parser.stack)-2], parser.stack[len(parser.stack)-1]]
        if len(parser.buffer) == 0:
          conf.append(-1)
        else:
          conf.append(parser.buffer[0])
      configurations.append([conf])

    return configurations

  def parsed_all(self, parsers):
    for parser in parsers:
      if not parser.is_tree_final():
        return False
    return True

  # In this function we select and perform the next move according to the scores obtained.
  # We need to be careful and select correct moves, e.g. don't do a shift if the buffer
  # is empty or a left arc if σ2 is the ROOT. For clarity sake we didn't implement
  # these checks in the parser so we must do them here. This renders the function quite ugly

  
  def parse_step(self,parsers, moves):
    moves_argm = moves.argmax(-1)
    parsers_to_update = np.logical_not([parser.is_tree_final() for parser in parsers])

    left_arc_mask = np.logical_and(moves_argm == 0, [parser.can_left_arc() for parser in parsers])
    right_arc_mask = moves_argm == 1
    shift_mask = np.logical_and(moves_argm == 2, [parser.can_shift() for parser in parsers])
    reduce_mask = np.logical_and(moves_argm == 3, [parser.can_reduce() for parser in parsers])

    for parser, left_arc, right_arc, shift, reduce in zip(parsers, left_arc_mask, right_arc_mask, shift_mask, reduce_mask):
        if left_arc:
            parser.left_arc()
        elif right_arc:
            parser.right_arc()
        elif shift:
            parser.shift()
        elif reduce:
            parser.reduce()


  def parse_step(self, parsers, moves):
    moves_argm = moves.argmax(-1)
    for i in range(len(parsers)):
        if parsers[i].is_tree_final():
            continue
        else:
              # Left arc
              if moves_argm[i] == 0 and parsers[i].can_left_arc(): 
                parsers[i].left_arc()
                  
              # Right arc
              elif moves_argm[i] == 1:
                parsers[i].right_arc()
                
              # Shift
              elif moves_argm[i] == 2 and parsers[i].can_shift():
                parsers[i].shift()
                
              # Reduce
              elif moves_argm[i] == 3 and parsers[i].can_reduce():
                parsers[i].reduce()
              
"""

'\n  def parse_step(self, parsers, moves):\n    moves_argm = moves.argmax(-1)\n    for i in range(len(parsers)):\n        if parsers[i].is_tree_final():\n            continue\n        else:\n              # Left arc\n              if moves_argm[i] == 0 and parsers[i].can_left_arc(): \n                parsers[i].left_arc()\n                  \n              # Right arc\n              elif moves_argm[i] == 1:\n                parsers[i].right_arc()\n                \n              # Shift\n              elif moves_argm[i] == 2 and parsers[i].can_shift():\n                parsers[i].shift()\n                \n              # Reduce\n              elif moves_argm[i] == 3 and parsers[i].can_reduce():\n                parsers[i].reduce()\n              \n'

In [90]:
# Evaluation
def evaluate(gold, preds):
  total = 0
  correct = 0

  for g, p in zip(gold, preds):
    for i in range(1,len(g)):
      total += 1
      if g[i] == p[i]:
        correct += 1

  return correct/total

# Training
def train(model, dataloader, criterion, optimizer):
  model.train()
  total_loss = 0
  count = 0

  for batch in dataloader:
    optimizer.zero_grad()
    sentences, paths, moves, trees = batch

    out = model(sentences, paths)
    labels = torch.tensor(sum(moves, [])).to(device) #sum(moves, []) flatten the array
    loss = criterion(out, labels)

    count +=1
    total_loss += loss.item()

    loss.backward()
    optimizer.step()

  return total_loss/count

# Testing
def test(model, dataloader):
  model.eval()

  gold = []
  preds = []

  for batch in dataloader:
    sentences, paths, moves, trees = batch
    with torch.no_grad():
      pred = model.infere(sentences)

      gold += trees
      preds += pred

  return evaluate(gold, preds)

In [91]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
model = Net(device)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)


for epoch in range(EPOCHS):
  avg_train_loss = train(model, train_dataloader, criterion, optimizer)
  val_uas = test(model, dev_dataloader)

  print("Epoch: {:3d} | avg_train_loss: {:5.3f} | dev_uas: {:5.3f} |".format( epoch, avg_train_loss, val_uas))

Device: cpu
Epoch:   0 | avg_train_loss: 1.117 | dev_uas: 0.479 |
Epoch:   1 | avg_train_loss: 0.951 | dev_uas: 0.539 |
Epoch:   2 | avg_train_loss: 0.925 | dev_uas: 0.580 |
Epoch:   3 | avg_train_loss: 0.911 | dev_uas: 0.600 |
Epoch:   4 | avg_train_loss: 0.900 | dev_uas: 0.610 |


KeyboardInterrupt: 

In [ ]:
test_uas = test(model, test_dataloader)
print("test_uas: {:5.3f}".format(test_uas))